<a href="https://colab.research.google.com/github/vvalerioluna/DataScience/blob/main/Cambio_Climatico_Cafe_Mexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto: Evaluación del Impacto del Cambio Climático en la Aptitud del Cultivo de Café en México

Este proyecto evalua y analiza las zonas aptas para el cultivo de café en el estado de Chiapas, considerando como el cambio climático podría afectar la idoneidad para el cultivo, utilizando técnicas de Machine Learning, Deep Learning en particular Random Forest y XGBoost/SVM, para clasificar y predecir zonas potencialmente aptas bajo el contexto del cambio climático y análisis geoespacial con datos abiertos del INEGI.

Estatus: En Desarrollo la fecha estimida de finalización Dic 2025.
Responsable Proyecto: Victor Valerio Luna
NOTA: El proyecto forma parte de una propuesta del Proyecto: El Potencial de las TIC en Sectores Emergentes en México y Latinoamérica del INFOTEC

## 1. Carga de librerías necesarias

In [ ]:
!pip install geopandas folium rasterio earthengine-api keplergl xarray netCDF4 tqdm --quiet


In [ ]:

import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from keplergl import KeplerGl
import matplotlib.pyplot as plt
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from shapely.geometry import Polygon
from shapely.geometry import Point

import seaborn as sns
import xarray as xr
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv2D, MaxPooling2D, Flatten
import requests
import json
from io import BytesIO
import zipfile
import warnings
warnings.filterwarnings("ignore")

## 2. Adquisición (Consulta) de datos geoespaciales y climáticos

In [ ]:
# montamos MyDrive
from google.colab import drive
drive.mount('/gdrive')
carpeta = 'RUTA/AL/DIRECTORIO/LOCAL/carpeta/9_DataScience/Portafolio/' #Ruta de archivos permanentes
RUTA_BASE = "/content/" # Ruta para archivo volatiles


In [ ]:
def descargar_zip(url, nombre_archivo):
    # Descarga un archivo ZIP desde una URL
    try:
        print(f"Descargando: {url}")
        respuesta = requests.get(url, stream=True)
        respuesta.raise_for_status()
        with open(nombre_archivo, 'wb') as f:
            for chunk in respuesta.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Ok Descarga completada: {nombre_archivo}")
    except requests.exceptions.RequestException as e:
        print(f"X Error en la descarga: {e}")
        return False
    return True

def descomprimir_zip(nombre_archivo, carpeta_destino="/content/"):
    # Descomprimir un archivo ZIP usando shell de Colab (más robusto que zipfile) realice pruebas y fue más óptimo
    try:
        print(f"Descomprimiendo {nombre_archivo}...")
        os.system(f'unzip -o "{nombre_archivo}" -d "{carpeta_destino}"')
        print("Ok Descompresión completada.")
        return True
    except Exception as e:
        print(f"X Error al descomprimir: {e}")
        return False

def buscar_geotiff(carpeta="/content/"):
    # Busca el primer archivo GeoTIFF en la carpeta
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(".tif"):
            return os.path.join(carpeta, archivo)
    return None

def visualizar_geotiff(path_tif, titulo="Visualización GeoTIFF"):
    # Carga y muestra una imagen GeoTIFF
    try:
        with rasterio.open(path_tif) as src:
            imagen = src.read(1)
            plt.figure(figsize=(10, 6))
            plt.imshow(imagen, cmap="viridis")
            plt.title(titulo)
            plt.colorbar(label="Valor")
            plt.axis("off")
            plt.show()
    except Exception as e:
        print(f"X Error al visualizar el GeoTIFF: {e}")

def procesar_descarga_visualizacion(url, nombre_zip, titulo_mapa):
    # Pipeline completo: descarga, descomprime, busca y visualiza
    if descargar_zip(url, nombre_zip):
        if descomprimir_zip(nombre_zip):
            path_tif = buscar_geotiff()
            if path_tif:
                print(f"!¡ Archivo GeoTIFF encontrado: {path_tif}")
                visualizar_geotiff(path_tif, titulo_mapa)
            else:
                print(" No se encontró ningún archivo .tif en la carpeta.")
        else:
            print(" No se pudo descomprimir el archivo ZIP.")
    else:
        print(" No se pudo descargar el archivo ZIP.")

# Procesamiento de archivos
url_uso_suelo = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/uso_suelo_veg/int_art/794551132210_t.zip"
procesar_descarga_visualizacion(url_uso_suelo, "uso_suelo.zip", "Uso de Suelo y Vegetación")

url_clima = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/CLIMAS/889463860136_s.zip"
procesar_descarga_visualizacion(url_clima, "clima.zip", "Índices de Extremos Climáticos")

## 4. Análisis exploratorio y visualización geográfica

In [ ]:

ruta_tif = buscar_geotiff("/gdrive/MyDrive/9_DataScience/Portafolio/794551132210_t")

# 1. Cargar y leer el GeoTIFF
with rasterio.open(ruta_tif) as src:
    uso_suelo_array = src.read(1)  # Solo una banda
    perfil = src.profile
    nodata = src.nodata
    print("Perfil del raster:")
    print(perfil)

# 2. Máscara de valores válidos (excluye nodata)
valores_validos = uso_suelo_array[uso_suelo_array != nodata]

# 3. Visualización general
plt.figure(figsize=(10, 6))
plt.imshow(uso_suelo_array, cmap="tab20", interpolation='none')
plt.colorbar(label="Clase de uso del suelo")
plt.title("Mapa de Uso del Suelo y Vegetación (IA, INEGI 2018)")
plt.axis("off")
plt.show()

# 4. Histograma de clases
conteo_clases = Counter(valores_validos.flatten())
clases = list(conteo_clases.keys())
frecuencias = list(conteo_clases.values())

plt.figure(figsize=(12, 6))
sns.barplot(x=clases, y=frecuencias, palette="tab20")
plt.xlabel("Código de clase de uso del suelo")
plt.ylabel("Frecuencia (número de píxeles)")
plt.title("Distribución de clases de uso del suelo")
plt.xticks(rotation=90)
plt.show()

# 5. Estadísticas básicas
print("\n Estadísticas básicas del raster:")
print(f"- Total de píxeles válidos: {len(valores_validos)}")
print(f"- Clases únicas encontradas: {sorted(clases)}")
print(f"- Clase más frecuente: {max(conteo_clases, key=conteo_clases.get)} con {max(frecuencias):,} píxeles")
print(f"- Clase menos frecuente: {min(conteo_clases, key=conteo_clases.get)} con {min(frecuencias):,} píxeles")

In [ ]:
# Definir ruta donde se descomprimió el archivo ZIP
carpeta_climas = "/gdrive/MyDrive/9_DataScience/Portafolio/889463860136_s/conjunto_de_datos/datos regionales"  # Asegúrate de haber descomprimido antes

# Buscar archivo .shp en la carpeta
def buscar_shp(carpeta):
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(".shp"):
            return os.path.join(carpeta, archivo)
    return None

shp_path = buscar_shp(carpeta_climas)

if shp_path:
    print(f"OK Archivo SHP encontrado: {shp_path}")

    # Leer archivo SHP con GeoPandas
    gdf = gpd.read_file(shp_path)

    # Primeras filas
    print(" Primeras filas del GeoDataFrame:")
    display(gdf.head())

    # Información general
    print("\n Información del GeoDataFrame:")
    print(gdf.info())

    # Tipo de geometrías y proyección
    print("\n Tipo de geometrías:", gdf.geom_type.unique())
    print(" CRS (proyección):", gdf.crs)

    # Columnas disponibles
    print("\n Columnas:")
    print(gdf.columns)

    # Estadísticas descriptivas
    print("\n Estadísticas descriptivas:")
    display(gdf.describe(include='all'))

    #  Visualización básica con matplotlib
    gdf.plot(figsize=(10, 8), column=gdf.columns[1], legend=True, cmap="viridis", edgecolor='black')
    plt.title("Mapa de Índices de Extremos Climáticos (INEGI)")
    plt.axis('off')
    plt.show()

    # Visualización interactiva con folium
    centro = gdf.geometry.centroid.unary_union.centroid.coords[:][0][::-1]
    m = folium.Map(location=centro, zoom_start=5)
    folium.GeoJson(gdf).add_to(m)
    m

else:
    print(" No se encontró archivo SHP en la carpeta. Verifica que esté descomprimido.")

In [ ]:
#1. Convertir ruta_tif a puntos georreferenciados (para Folium / Kepler)
def raster_a_puntos(ruta_tif, max_muestras=5000):
    with rasterio.open(ruta_tif) as src:
        banda = src.read(1)
        transform = src.transform
        nodata = src.nodata

        puntos = []
        for row in range(banda.shape[0]):
            for col in range(banda.shape[1]):
                valor = banda[row, col]
                if valor != nodata:
                    x, y = transform * (col, row)
                    puntos.append({"geometry": Point(x, y), "valor": valor})
        print(f"OK Puntos extraídos: {len(puntos)}")

        # Reducir cantidad para visualización (opcional)
        if max_muestras and len(puntos) > max_muestras:
            puntos = np.random.choice(puntos, max_muestras, replace=False).tolist()

        gdf_puntos = gpd.GeoDataFrame(puntos, crs=src.crs)
        return gdf_puntos

gdf_raster_pts = raster_a_puntos(ruta_tif)
display(gdf_raster_pts.head())


In [ ]:
#2. Convertir gdf (clima) a GeoJSON para usar en Folium y Kepler
geojson_clima = json.loads(gdf.to_json())
geojson_raster = json.loads(gdf_raster_pts.to_json())


In [ ]:
#3. Visualizar en Folium
# Obtener centro del mapa
centro = gdf_raster_pts.unary_union.centroid.coords[0][::-1]

m = folium.Map(location=centro, zoom_start=6)

# Agregar capa raster como puntos
folium.GeoJson(geojson_raster, name="Uso de Suelo",
               tooltip=folium.GeoJsonTooltip(fields=["valor"], aliases=["Clase:"])
              ).add_to(m)

# Agregar capa climática
folium.GeoJson(geojson_clima, name="Clima Extremo",
               tooltip=folium.GeoJsonTooltip(fields=gdf.columns[1:3].tolist())
              ).add_to(m)

folium.LayerControl().add_to(m)

m


In [ ]:
#4. Visualizar en Kepler.gl
mapa_kepler = KeplerGl(height=600)

mapa_kepler.add_data(data=gdf_raster_pts, name="Uso de Suelo (puntos)")
mapa_kepler.add_data(data=gdf, name="Clima Extremo")

mapa_kepler


## 5. Modelado con Machine Learning (En proceso ya que se validará)
Criterios de Aptitud para el Cultivo de Café en Chiapas
1. Temperatura
Óptima: Entre 18°C y 24°C.
Límite Inferior: No menos de 15°C.
Límite Superior: No más de 30°C.
Impacto: Temperaturas fuera de este rango pueden afectar negativamente la floración y el desarrollo de los granos.
2. Precipitación
Óptima: Entre 1,500 mm y 2,500 mm anuales.
Distribución: Idealmente bien distribuida a lo largo del año, con una estación seca corta para favorecer la floración.
3. Altitud
Óptima para Coffea arabica: Entre 1,000 y 2,000 metros sobre el nivel del mar (msnm).
Óptima para Coffea canephora (Robusta): Entre 200 y 800 msnm.
Impacto: La altitud influye en la temperatura y la humedad, afectando la calidad del café.
4. Tipo de Suelo
Textura: Suelos franco-arenosos a franco-arcillosos.
pH: Entre 5.0 y 6.5.
Drenaje: Bien drenados, con buena capacidad de retención de agua.
Fertilidad: Ricos en materia orgánica y nutrientes esenciales como nitrógeno, fósforo y potasio.
Variedades de Café Más Vendidas
Coffea arabica: Incluye variedades como Bourbon, Typica, Caturra, y Geisha. Estas variedades son apreciadas por su sabor suave y aromático.
Coffea canephora (Robusta): Conocida por su mayor resistencia a enfermedades y su contenido más alto de cafeína, aunque su sabor es más fuerte y amargo 2.
Consideraciones Adicionales
Cambio Climático: Es importante considerar las proyecciones climáticas para los años 2026 y 2027, ya que el cambio climático puede alterar las condiciones óptimas actuales. Adaptar las prácticas agrícolas y seleccionar variedades resistentes será crucial.


In [ ]:
# ERROR: esta celda requiere revisión (ruta o variable no definida)
# Random Forest para modelar idoneidad del cultivo
# Datos simulados como ejemplo
X = datos_climaticos[["temp", "precip"]]
y = datos_suelo["cafe"]  # binario: café sí o no
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
modelo_rf = RandomForestClassifier()
modelo_rf.fit(X_train, y_train)
print(classification_report(y_test, modelo_rf.predict(X_test)))


In [ ]:
np.random.seed(42)
datos_climaticos = pd.DataFrame({
    'temp': np.random.normal(loc=22, scale=3, size=1000),
    'precip': np.random.normal(loc=1500, scale=300, size=1000)
})
datos_suelo = pd.DataFrame({
    'cafe': np.random.choice([0, 1], size=1000)
})

##  6. Modelado con Deep Learning (LSTM o CNN)

In [ ]:
X = datos_climaticos[["temp", "precip"]]
y = datos_suelo["cafe"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_test)
print(" Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:

# Ejemplo de red neuronal densa simple
#modelo_dl = Sequential([
#    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#    Dense(32, activation='relu'),
#    Dense(1, activation='sigmoid')
#])
#modelo_dl.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#modelo_dl.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


## 7. Evaluación y escenarios futuros
- Comparar métricas entre modelos
- Proyección de cambios bajo escenarios RCP 4.5 y 8.5

In [ ]:
modelo_dl = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
modelo_dl.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento
modelo_dl.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


## 8. Visualización final en mapas interactivos

In [ ]:
# Mostrar predicciones de idoneidad en un mapa
# (reconstrucción de capas raster con predicciones)


In [ ]:
print(" Fuentes sugeridas:")
print("- INEGI Uso del Suelo: https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=889463197248")
print("- WorldClim (clima histórico y futuro): https://www.worldclim.org/data/worldclim21.html")
print("- CONABIO Geoportal: https://www.conabio.gob.mx/informacion/gis/")

##  9. Fuentes de datos
- [INEGI Uso de Suelo y Vegetación](https://www.inegi.org.mx/app/biblioteca/ficha.html?upc=889463197248)
- [WorldClim v2](https://www.worldclim.org/data/worldclim21.html)
- [CONABIO Geoportal](https://www.conabio.gob.mx/informacion/gis/)
- [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/)

## Conclusiones

- Se propone una metodología replicable para evaluar la aptitud agrícola en función de cambios climáticos.
- Para que este notebook funcione correctamente, asegúrate de incluir los archivos GeoTIFF, NetCDF y shapefiles necesarios en el repositorio.
- Las rutas han sido generalizadas para facilitar su uso en diferentes entornos.
